# Tensor与module的保存与加载

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [1]:
import torch
from torch import nn
import os

## Tensor的保存与加载

In [3]:
# 序列化到文件
t = torch.randn(5, 5)
print(t)
torch.save(t, "/tmp/t.tensor")
t1 = torch.load("/tmp/t.tensor", weights_only=True)
assert torch.sum(t - t1) < 1e-5
os.remove("/tmp/t.tensor")

tensor([[-0.7775, -1.1418,  2.3894,  1.1619, -0.3040],
        [-0.3100,  0.7757, -0.9492, -0.7231, -1.1211],
        [ 0.0469,  0.5327, -1.4517,  0.1013,  0.3704],
        [-0.9112, -0.1610,  0.9753,  0.5036, -1.0410],
        [ 1.7074, -1.2069,  0.5206,  1.0643, -1.1562]])


保存到内存中，以及从内存中加载

In [5]:
from io import BytesIO

t = torch.randn(5, 5)
buffer = BytesIO()
torch.save(t, buffer)
# 重置buffer的读写位置
buffer.seek(0)
t1 = torch.load(buffer, weights_only=True)

## Module的保存与加载

可以单独保存模型的参数，也可以把整个模型保存起来

In [6]:
model = nn.Sequential(nn.Linear(25, 100), nn.ReLU(), nn.Linear(100, 10))
torch.save(model.state_dict(), "/tmp/mlp-params.pt")

In [8]:
params = torch.load("/tmp/mlp-params.pt", weights_only=True)
# 现在的模型加载一份离线的参数
model.load_state_dict(params)

<All keys matched successfully>

直接保存整个 Moduel

In [9]:
torch.save(model, "/tmp/mlp-model.pt")

In [12]:
print(torch.load("/tmp/mlp-model.pt", weights_only=False))

Sequential(
  (0): Linear(in_features=25, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)


在 PyTorch 中，如果你直接保存整个模型模块（即调用 `torch.save(model, 'model.pth')`），它会尝试保存模型的所有内容，包括模型的结构、参数以及模型定义中用到的所有第三方库的引用。然而，这种方法并不保存第三方库的实际实现代码，只保存了对这些库的引用和调用。因此，当你在不同的环境中加载这个模型时，必须确保所有依赖的第三方库已经安装且版本兼容，否则可能会遇到问题。

仅保存 state_dict 可以确保模型在不同版本的 PyTorch 中更容易兼容。保存整个模型模块可能会导致在 PyTorch 更新版本后无法加载旧版本模型的问题，因为整个模型包含了版本相关的信息和代码。

## 使用GPU

In [13]:
gpu_model = model.to(device="cuda:0")

In [14]:
input = torch.randn(1, 25, device="cuda:0")
gpu_model(input)

tensor([[-0.5465, -0.3952, -0.3197, -0.1548, -0.4683, -0.0815, -0.5872, -0.6991,
          0.1828,  0.1191]], device='cuda:0', grad_fn=<AddmmBackward0>)

GPU下保存的Tesnor或model，加载回来时，还是在对应的GPU上

In [16]:
t = torch.randn(3, 4, device="cuda")
torch.save(t, "/tmp/t-cuda.pt")
t = torch.load("/tmp/t-cuda.pt", weights_only=True)
t.device

device(type='cuda', index=0)

## 优化器状态的保存与加载

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}

torch.save(checkpoint, "/tmp/tmp.ckpt")

In [18]:
checkpoint = torch.load("/tmp/tmp.ckpt", weights_only=True)
print(checkpoint.keys())

dict_keys(['state_dict', 'optimizer'])


In [19]:
model.load_state_dict(checkpoint["state_dict"])
optimizer.load_state_dict(checkpoint["optimizer"])

## `torch.load`与`torch.save`的原理

在 PyTorch 中，`torch.save` 和 `torch.load` 是用于模型、张量和其他任意数据的序列化和反序列化的函数。它们底层基于 Python 的 pickle 模块，并通过一些优化来确保高效、灵活地保存和加载数据。


### Save

torch.save 实际上是将对象转换为字节流，即序列化。这是通过 Python 的 pickle 模块完成的。PyTorch 对 pickle 进行了扩展，使其能够处理张量等 PyTorch 特有的对象。

* 如果保存的对象包含张量，PyTorch 通过自定义的序列化机制将这些张量序列化。
* PyTorch 内部会识别出张量的设备信息（如 CPU 或 GPU）以及张量的数据类型，并确保这些信息在序列化时被保留。
* 对于张量，`torch.save` 会对其进行压缩或使用优化的存储方式，以便节省磁盘空间。
* 为了增强向后兼容性，PyTorch 还会保存版本信息，这样在不同版本的 PyTorch 中使用时，仍然可以正常加载模型和数据。

### Load

torch.load 会读取文件中的字节流，并使用 pickle 反序列化为原始的 Python 对象。如果反序列化过程中涉及到张量，PyTorch 会恢复张量的设备和数据类型。通过 pickle，PyTorch 内部的序列化扩展会识别并恢复保存时的张量对象。

在加载模型时，PyTorch 会自动将保存时的设备信息（如 CPU 或 GPU）与当前设备进行匹配。如果当前环境中没有 GPU，但保存时模型是在 GPU 上，PyTorch 可以通过 map_location 参数将张量映射到 CPU 上。

In [21]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(100, 50)
        self.layer2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

# 保存模型的 state_dict（假设已经训练过）
model = MyModel()
torch.save(model.state_dict(), '/tmp/model.pth')  # 已经保存过，不需要重复保存


# 定义自定义的 map_location 函数，将不同的层加载到不同的设备
def custom_map_location(storage, loc):
    if "cuda:0" in loc:  # 如果原来在 cuda:0 上
        return storage.cuda(0)  # 保持在 cuda:0
    elif "cuda:1" in loc:  # 如果原来在 cuda:1 上
        return storage.cuda(1)  # 保持在 cuda:1
    else:
        return storage.cpu()  # 其余情况移动到 CPU


# 使用自定义的 map_location 函数加载模型
state_dict = torch.load("/tmp/model.pth", map_location=custom_map_location, weights_only=True)

os.remove("/tmp/model.pth")